# Network of Businesses based on Sequences

In [1]:
# Import necessary libraries
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
import collections
from itertools import combinations
import networkx as nx
from glob import glob
from networkx.algorithms import community
%matplotlib inline
from matplotlib import *
import graph_tool.all as gt
import math
import datetime

## Edges: User's Sequenes of Services Visited

# DO VEGAS

In [2]:
Yelp_Reviews = pd.read_csv("HOPEFUL.csv")

In [3]:
Yelp_Reviews = Yelp_Reviews[['business_id','date','user_id','business_catgrylv2'\
                             ,'business_catgrylv1','stars','business_num_reviews'\
                            , 'business_average_stars']]

In [4]:
Yelp_Reviews = Yelp_Reviews[Yelp_Reviews['stars']>3]

In [5]:
Yelp_Reviews.head(2)

,business_id,date,user_id,business_catgrylv2,business_catgrylv1,stars,business_num_reviews,business_average_stars
0,0W4lkclzZThpx3V65bVgig,2016-05-28,bv2nCi5Qv5vroFiqKGopiw,Bars,Breakfast & Brunch,5,1953,4.0
1,AEx2SYEUJmTxVVB18LlCwA,2016-05-28,bv2nCi5Qv5vroFiqKGopiw,Bars,Food,5,84,4.0


In [6]:
Yelp_BusinessCats1 = dict(zip(Yelp_Reviews.business_id,Yelp_Reviews.business_catgrylv1))

In [7]:
Yelp_BusinessCats2 = dict(zip(Yelp_Reviews.business_id,Yelp_Reviews.business_catgrylv2))

In [8]:
Yelp_Businesses = pd.read_json('YelpDataset/business.json',lines=True)

In [9]:
Yelp_BusinessesCharlotte = Yelp_Businesses[Yelp_Businesses['city'] == 'Charlotte']
Yelp_BusinessesCharlotte.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
14,600 E 4th St,"{'GoodForMeal': {'dessert': False, 'latenight'...",fNMVV_ZX7CJSDWQGdOM8Nw,"[Restaurants, American (Traditional)]",Charlotte,"{'Friday': '7:00-15:00', 'Tuesday': '7:00-15:0...",1,35.221647,-80.839345,Showmars Government Center,Uptown,28202,7,3.5,NC
56,"1225 S Church St, Ste B",{'GoodForKids': False},ykJM7EuGziATZ5u2qIT08g,"[Active Life, Fitness & Instruction, Gyms]",Charlotte,"{'Monday': '5:00-17:00', 'Tuesday': '5:00-17:0...",1,35.220211,-80.855785,My Fitness World,South End,28203,4,5.0,NC
77,8540 E Independence Blvd,"{'BusinessAcceptsCreditCards': True, 'Restaura...",NFTh6mj3X2AnHJCKIFUk5Q,"[Hotels, Event Planning & Services, Hotels & T...",Charlotte,{},1,35.147422,-80.726487,Fairfield Inn & Suites,,28105,17,4.0,NC
84,"5110 Park Rd, Ste 2F","{'DogsAllowed': True, 'BusinessParking': {'gar...",nJ3mXjItS8WcwhYbzbfDQw,"[Watch Repair, Jewelry Repair, Shopping, Local...",Charlotte,"{'Monday': '10:00-18:00', 'Tuesday': '10:00-18...",1,35.160889,-80.849260,Scott's Karat Patch,South Park,28209,5,4.0,NC
97,"1816 Galerea Blvd, Ste D","{'RestaurantsTakeOut': True, 'NoiseLevel': 'qu...",HAX1zec191t7QkT2sBZ76A,"[Restaurants, Cuban]",Charlotte,{},0,35.137223,-80.734594,La Isla Cuban Restaurant,,28270,4,3.0,NC


In [10]:
Yelp_BusinessesCharlotteNAMES = dict(zip(Yelp_BusinessesCharlotte.business_id,Yelp_BusinessesCharlotte.name))


In [11]:
busCharlotteSet = set(Yelp_BusinessesCharlotte['business_id'])
def businessInCharlotte(business):
    if business in busCharlotteSet: return "Charlotte"
    else: return "NOT_Charlotte"

Yelp_Reviews['city'] = Yelp_Reviews['business_id'].apply(businessInCharlotte)
Yelp_Reviews = Yelp_Reviews[Yelp_Reviews['city'] == 'Charlotte']

In [12]:
Yelp_ReviewsCharlotte = Yelp_Reviews

In [13]:
Yelp_ReviewsCharlotte

,business_id,date,user_id,business_catgrylv2,business_catgrylv1,stars,business_num_reviews,business_average_stars,city
631,sPwgafejYVdf8hd4-oDYVg,2015-04-27,ewoVtCaCosO1dLqlPfuzNQ,Bars,Breakfast & Brunch,4,199,4.0,Charlotte
632,R1jJQi2yR44D_2ileqr8kA,2016-06-02,ewoVtCaCosO1dLqlPfuzNQ,Bars,Sandwiches,5,315,4.5,Charlotte
633,RcB7STNxoSDjtoK1pXgWBg,2016-05-24,ewoVtCaCosO1dLqlPfuzNQ,Bars,Breakfast & Brunch,4,285,4.0,Charlotte
634,r_oCBNif2W8mnSXnrXMwPQ,2016-05-24,ewoVtCaCosO1dLqlPfuzNQ,Fashion,Food,5,5,4.0,Charlotte
635,AuIiqDeL65kczmiFSkAWrA,2015-04-27,ewoVtCaCosO1dLqlPfuzNQ,Restaurants,Food,4,281,3.5,Charlotte
636,npWIa3dIPEV1MTaZNq6AUA,2015-04-27,ewoVtCaCosO1dLqlPfuzNQ,Bars,American (Traditional),5,261,4.0,Charlotte
1138,M0pkmBUi_CI0qrzN7ee80Q,2016-11-17,Ma8xZz6Myj1EMvGqs4-KAQ,Bars,Bars,5,374,4.0,Charlotte
1141,T2tEMLpTeSMxLKpxwFdS3g,2016-11-17,Ma8xZz6Myj1EMvGqs4-KAQ,Bars,Breakfast & Brunch,5,945,4.0,Charlotte
1142,RAh9WCQAuocM7hYM5_6tnw,2016-11-17,Ma8xZz6Myj1EMvGqs4-KAQ,Bars,Breakfast & Brunch,5,1345,4.0,Charlotte
1144,gChmBjLSe3qa8rzEngoDLQ,2017-10-31,Ma8xZz6Myj1EMvGqs4-KAQ,Bars,American (Traditional),4,523,3.0,Charlotte


In [14]:
Yelp_ReviewsCharlotte.groupby('user_id').agg('count').index

Index(['--0RtXvcOIE4XbErYca6Rw', '--3WaS23LcIXtxyFULJHTA',
       '--5-MQZPsn2CJkyMFQkOgQ', '--6D_IuxyKTN53pHi904ag',
       '--6Ke7_lBBM6XAramtPoWw', '--GlFajt0A35it7U3mxgqQ',
       '--OZO4c_vDOVC2z2BWr92A', '--PgJdSVUpp7uCr99nUXqw',
       '--ZskqYaXUI0KusYUv-wYQ', '--i7WS2pwOQKQl6uB15q1A',
       ...
       'zzDbtZtiMhGGi5BZhKf1BQ', 'zzF6P7epT9hIbglz0-pKwA',
       'zzQQEMYy2rdeVfiBgVJ3ng', 'zzYF64kl9es4c16fv68OIQ',
       'zza-gQ2jzjAAJkalUHuO6Q', 'zzawiLRYd4TjHKVyx6zjUQ',
       'zzg3bpldFxSyDJ4s1nMNvw', 'zzpGgInpynAC9do60jTAsg',
       'zzvm_VD3HjU6RMx8zq7g1g', 'zzw0Z6-_VDp9ShIRSKIsQw'],
      dtype='object', name='user_id', length=53863)

In [15]:
Yelp_ReviewsCharlotte.groupby('user_id').agg('count')['date'].value_counts()

1      33895
2       8371
3       3760
4       1986
5       1287
6        859
7        612
8        495
9        351
10       277
11       230
12       180
13       156
14       122
15       110
16        97
17        91
18        78
19        61
20        57
21        49
22        43
26        41
23        32
25        28
24        28
32        26
29        23
28        23
27        21
       ...  
246        1
188        1
146        1
91         1
125        1
139        1
107        1
109        1
301        1
92         1
106        1
110        1
137        1
207        1
239        1
335        1
104        1
121        1
127        1
145        1
327        1
177        1
241        1
155        1
114        1
262        1
198        1
134        1
102        1
159        1
Name: date, Length: 155, dtype: int64

In [16]:
Yelp_ReviewsCharlotte = Yelp_ReviewsCharlotte.sort_index(by = 'date')

/opt/condaenvs/jupyterhub/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


In [17]:
Yelp_ReviewsCharlotte = Yelp_ReviewsCharlotte.reset_index()

In [18]:
list_of_lists = []
def create_lists():
    temp_user = Yelp_ReviewsCharlotte['user_id'][0]
    temp_list = []
    for x in range(0, len(Yelp_ReviewsCharlotte) - 1):
        if (temp_user == Yelp_ReviewsCharlotte['user_id'][x]):
            # Append business id this current temp list
            temp_tuple = (Yelp_ReviewsCharlotte['business_id'][x], Yelp_ReviewsCharlotte['date'][x])
            temp_list.append(temp_tuple)
        elif (temp_user != Yelp_ReviewsCharlotte['user_id'][x]):
            # Append the current temp list to list of lists
            list_of_lists.append(temp_list)
            # Clear temp list
            temp_list = []
            # Add this value to the new temp list
            temp_tuple = (Yelp_ReviewsCharlotte['business_id'][x], Yelp_ReviewsCharlotte['date'][x])
            temp_list.append(temp_tuple)
            # Make the new user the temp user
            temp_user = Yelp_ReviewsCharlotte['user_id'][x]

In [19]:
create_lists()

In [20]:
print(list_of_lists[:3])

[[('2Q2uE1dwhqMFOE5CYfvMEQ', '2004-10-25')], [('ydUqgWsF3F27TbauOyib0w', '2004-12-19')], [('nMo0ozHO7cqsPaxw_7N6ow', '2005-06-17'), ('osSwv6CJy5hDKQdOKeyTow', '2005-06-17'), ('ZCXjNG1EBFiKKMtR9DOTGg', '2005-06-17'), ('Oy2WnPyiOlPFvPKMIuOC8w', '2005-06-18'), ('8q_q76SAbCLeAbZsW7oAFQ', '2005-06-18'), ('KEGLWeFAWXvo0W2LnujhtQ', '2005-07-24')]]


In [21]:
newListOfLists = []
for i in range(len(list_of_lists)):
    if len(list_of_lists[i]) != 1:
        newListOfLists.append(list_of_lists[i])

In [22]:
# Put only the business info into a new list of lists
temp_var = []
id_only = []
for x in range(0, len(newListOfLists) - 1):
    temp_var = [item[0] for item in newListOfLists[x]]
    id_only.append(temp_var)

In [23]:
# Take each pair and make it into a tuple
businesses_tuple = []

for y in range(0, len(id_only) - 1):
    for x in range(0, len(id_only[y]) - 1):
        businesses_tuple.append((tuple(id_only[y][x:(x+2)])))

In [24]:
businessesEdgesDf = pd.Series(businesses_tuple).value_counts().to_frame('weight').reset_index()

In [25]:
def makeEdgeWithWeight(row):
    edge = row['index']
    weight = row['weight']
    edge = edge + ({'weight':weight},)
    return edge

businessesEdgesDf['newEdges'] = businessesEdgesDf.apply(makeEdgeWithWeight,axis = 1)

In [26]:
businessesEdgesDf['newEdges'][123]

('wiEQrYSNofc10q7Pm26b0w', 'MUf-xrAxrliNQ4IvOeQ9oQ', {'weight': 1})

In [27]:
businesses_tupleNEW = list(businessesEdgesDf['newEdges'])

In [28]:
### Edges stored in " businesses_tuple "

## Vertices: Business ID's

In [29]:
# Put business id's into a list to represent the vertices
Business_IDs = list(busCharlotteSet)

In [30]:
# Ensure the correct information has been appended
print(Business_IDs[:5])

['_xYM9osYumECb42IzSk-tA', 'uyH2PBvordYWCZ8FyrJH8A', 'nUgd6F-xIFdGSyQbzRojWQ', 'uPY5oatcnkBa6MFnUQHFCw', 'qs-o8DFpkPNoUSfy5IPIFg']


In [31]:
# Check the number of nodes
print(len(Business_IDs))

8553


In [32]:
### Vertices or nodes stored in " Business_IDs "

## Creating the Network

In [33]:
G=nx.DiGraph()

In [34]:
# Add Nodes
G.add_nodes_from(Business_IDs)

In [35]:
# Add Edges
#  G.add_edges_from([(1,2,{'color':'blue'}), (2,3,{'weight':8})])

G.add_edges_from(businesses_tupleNEW)
print('edges added')

edges added


In [36]:
G.number_of_nodes()

8553

In [37]:
G.number_of_edges()

6321

In [38]:
nx.set_node_attributes(G, Yelp_BusinessCats1, "Category_Level_1")
nx.set_node_attributes(G, Yelp_BusinessCats2, "Category_Level_2")

In [39]:
cat1 = nx.get_node_attributes(G,"Category_Level_1")
cat2 = nx.get_node_attributes(G,"Category_Level_2")

In [40]:
for node in list(G.nodes()):
    if node in cat2:
        localList = []
        for neighbor in G.successors(node):
            if neighbor in cat2:
                localList.append(cat2[neighbor])
        print((cat2[node]),localList)
        



Fashion ['Bars', 'Restaurants']
Financial Services []
Home & Garden []
Bars ['Bars']
Restaurants ['Bars']
Restaurants ['Restaurants', 'Bars', 'Bars', 'Pubs', 'Hair Removal']
Home & Garden []
Bars []
Bars ['Bars', 'Bars', 'Restaurants', 'Bars', 'Pubs', 'Restaurants', 'Bars', 'Pubs', 'Restaurants', 'Restaurants', 'Bars']
Home Cleaning []
Restaurants []
Oil Change Stations []
Restaurants []
Beauty & Spas ['Restaurants']
Bars []
Home & Garden []
Home & Garden []
Bars []
Restaurants ['Restaurants']
Hair Removal []
Financial Services []
Restaurants ['Bars', 'Restaurants', 'Bars']
Home & Garden ['Bars']
Bars ['Restaurants']
Hair Removal []
Bars ['Bars']
Bars ['Cosmetics & Beauty Supply']
Bars []
Home Cleaning []
Oil Change Stations ['Bars']
Fashion ['Pubs']
Bars ['Bars']
Beauty & Spas []
Bars []
Beauty & Spas []
Bars []
Bars ['Bars', 'Beauty & Spas', 'Bars', 'Bars', 'Sports Clubs', 'Bars', 'Active Life', 'Bars', 'Restaurants', 'Sports Clubs', 'Bars', 'Bars', 'Bars', 'Restaurants', 'Bars', 'Be

Beauty & Spas []
Bars []
Bars ['Restaurants', 'Oil Change Stations', 'Pubs', 'Restaurants', 'Restaurants', 'Bars', 'Bars']
Fashion []
Home & Garden []
Fashion []
Fashion []
Active Life []
Bars []
Restaurants ['Pubs']
Bars ['Bars']
Restaurants ['Fashion']
Bars []
Bars ['Bars', 'Bars', 'Bars', 'Restaurants', 'Bars', 'Pubs', 'Restaurants']
Beauty & Spas []
Pubs ['Restaurants', 'Bars', 'Restaurants']
Beauty & Spas []
Fashion []
Home Cleaning []
Hair Removal []
Restaurants []
Beauty & Spas []
Home Cleaning []
Bars ['Bars', 'Bars']
Bars []
Restaurants []
Bars []
Restaurants []
Fashion []
Beauty & Spas []
Bars ['Restaurants']
Beauty & Spas []
Bars []
Cosmetics & Beauty Supply []
Fashion []
Active Life []
Beauty & Spas []
Bars []
Bars ['Restaurants']
Restaurants []
Beauty & Spas ['Bars', 'Used', 'Bars']
Home Cleaning []
Active Life ['Active Life', 'Bars']
Bars []
Bars []
Restaurants []
Active Life []
Bars []
Beauty & Spas []
Beauty & Spas []
Bars []
Fashion ['Fashion']
Beauty & Spas []
Home & 

Fashion []
Bars []
Beauty & Spas []
Bars ['Restaurants', 'Bars', 'Bars', 'Restaurants']
Home & Garden ['Restaurants']
Bars []
Bars []
Bars ['Bars', 'Restaurants', 'Bars', 'Bars', 'Bars', 'Home & Garden', 'Bars', 'Bars', 'Restaurants', 'Bars', 'Bars', 'Bars']
Bars []
Home & Garden []
Bars []
Active Life []
Restaurants []
Home & Garden []
Home & Garden []
Bars []
Home & Garden []
Pubs ['Restaurants']
Restaurants ['Bars', 'Bars', 'Bars', 'Used']
Beauty & Spas []
Restaurants []
Fashion []
Home & Garden []
Bars []
Bars ['Beauty & Spas']
Oil Change Stations []
Restaurants []
Fashion []
Beauty & Spas []
Restaurants ['Restaurants', 'Bars', 'Active Life', 'Bars', 'Bars']
Home & Garden []
Restaurants []
Beauty & Spas []
Restaurants []
Bars []
Home & Garden []
Bars ['Restaurants']
Bars []
Beauty & Spas []
Beauty & Spas []
Bars []
Restaurants []
Restaurants []
Bars ['Restaurants', 'Bars', 'Bars', 'Bars', 'Bars', 'Bars', 'Restaurants', 'Pubs']
Home & Garden []
Restaurants []
Bars ['Bars', 'Bars', '

Restaurants []
Beauty & Spas []
Bars []
Hair Removal []
Hair Removal []
Beauty & Spas []
Restaurants []
Home & Garden []
Active Life []
Home Cleaning []
Home & Garden ['Bars', 'Bars']
Beauty & Spas ['Restaurants']
Home & Garden []
Beauty & Spas []
Active Life []
Pubs []
Fashion ['Fashion', 'Restaurants']
Used []
Oil Change Stations []
Bars []
Bars []
Fashion []
Bars []
Fashion []
Restaurants []
Active Life []
Beauty & Spas []
Pubs ['Restaurants', 'Fashion', 'Bars', 'Restaurants', 'Bars', 'Restaurants']
Home & Garden []
Fashion []
Bars []
Financial Services []
Restaurants []
Bars ['Bars']
Beauty & Spas []
Active Life ['Bars', 'Restaurants']
Restaurants []
Bars []
Bars ['Active Life']
Beauty & Spas []
Restaurants []
Bars []
Home & Garden []
Bars ['Restaurants']
Home & Garden []
Restaurants ['Bars', 'Cosmetics & Beauty Supply']
Beauty & Spas []
Restaurants []
Cosmetics & Beauty Supply []
Restaurants []
Bars []
Bars []
Beauty & Spas []
Home & Garden []
Bars []
Bars ['Bars']
Used []
Bars ['

Beauty & Spas ['Restaurants', 'Fashion']
Bars []
Restaurants []
Restaurants []
Active Life []
Fashion []
Bars []
Home & Garden []
Restaurants ['Bars', 'Hair Removal']
Used []
Restaurants ['Bars']
Bars ['Bars']
Beauty & Spas []
Home Cleaning []
Restaurants []
Beauty & Spas ['Bars']
Beauty & Spas []
Used []
Restaurants []
Restaurants []
Fashion []
Home & Garden []
Fashion []
Bars ['Restaurants']
Bars ['Bars', 'Bars', 'Bars', 'Bars', 'Restaurants', 'Bars', 'Pubs', 'Restaurants', 'Bars', 'Bars', 'Bars', 'Bars', 'Bars', 'Bars', 'Restaurants']
Restaurants []
Beauty & Spas []
Home & Garden []
Home & Garden []
Bars ['Bars']
Bars ['Bars']
Fashion []
Bars ['Bars']
Bars ['Restaurants', 'Bars', 'Restaurants', 'Bars', 'Restaurants', 'Restaurants', 'Restaurants', 'Bars', 'Restaurants', 'Hair Removal', 'Bars', 'Bars', 'Bars', 'Bars']
Fashion []
Beauty & Spas []
Restaurants ['Beauty & Spas', 'Restaurants']
Restaurants ['Beauty & Spas', 'Bars', 'Restaurants', 'Beauty & Spas']
Home & Garden []
Beauty & 

Pubs []
Active Life ['Beauty & Spas']
Fashion []
Beauty & Spas []
Restaurants []
Home & Garden []
Beauty & Spas []
Oil Change Stations []
Restaurants []
Restaurants ['Bars']
Restaurants []
Cosmetics & Beauty Supply []
Home & Garden []
Restaurants []
Bars []
Bars ['Restaurants']
Oil Change Stations []
Beauty & Spas []
Fashion []
Used []
Restaurants []
Beauty & Spas []
Restaurants ['Bars']
Restaurants []
Restaurants []
Bars ['Bars', 'Bars', 'Bars', 'Pubs', 'Bars', 'Restaurants', 'Bars', 'Bars', 'Restaurants', 'Bars', 'Bars', 'Bars', 'Restaurants', 'Bars', 'Restaurants', 'Restaurants', 'Beauty & Spas', 'Bars', 'Bars', 'Bars', 'Bars', 'Bars', 'Bars', 'Bars', 'Bars', 'Restaurants', 'Restaurants', 'Restaurants', 'Restaurants', 'Bars']
Beauty & Spas []
Bars []
Restaurants []
Beauty & Spas []
Restaurants []
Restaurants []
Restaurants []
Restaurants []
Fashion []
Home & Garden ['Bars']
Home & Garden []
Restaurants []
Hair Removal []
Restaurants []
Bars []
Home & Garden []
Bars ['Bars', 'Bars', 

Bars []
Bars ['Bars']
Beauty & Spas []
Restaurants ['Restaurants']
Restaurants []
Home & Garden []
Used []
Beauty & Spas []
Bars ['Bars', 'Fashion', 'Bars', 'Restaurants']
Home & Garden []
Home & Garden []
Beauty & Spas []
Fashion []
Fashion ['Cosmetics & Beauty Supply', 'Pubs', 'Bars', 'Bars', 'Pubs']
Restaurants []
Restaurants ['Bars']
Bars ['Restaurants']
Bars ['Bars', 'Bars', 'Bars', 'Restaurants']
Bars []
Bars []
Financial Services []
Restaurants []
Used []
Beauty & Spas []
Home Cleaning []
Restaurants []
Beauty & Spas []
Used ['Restaurants', 'Restaurants', 'Restaurants', 'Fashion', 'Fashion']
Fashion []
Restaurants []
Bars []
Restaurants []
Restaurants ['Restaurants', 'Bars']
Home & Garden []
Fashion []
Bars ['Restaurants', 'Bars']
Oil Change Stations []
Beauty & Spas []
Restaurants []
Beauty & Spas []
Beauty & Spas []
Beauty & Spas []
Hair Removal []
Pubs []
Beauty & Spas []
Home & Garden []
Sports Clubs ['Fashion', 'Restaurants', 'Fashion']
Bars []
Restaurants ['Restaurants']
H

Home Cleaning []
Beauty & Spas []
Home & Garden []
Bars []
Bars ['Beauty & Spas', 'Pubs', 'Bars', 'Restaurants', 'Restaurants', 'Restaurants']
Bars []
Hair Removal []
Beauty & Spas ['Bars']
Home & Garden []
Home & Garden []
Cosmetics & Beauty Supply ['Restaurants']
Pubs []
Bars []
Used []
Bars ['Beauty & Spas', 'Restaurants']
Restaurants []
Restaurants []
Bars []
Hair Removal []
Beauty & Spas []
Restaurants []
Cosmetics & Beauty Supply []
Home & Garden []
Bars []
Bars ['Restaurants', 'Bars', 'Restaurants', 'Restaurants', 'Fashion']
Bars ['Bars', 'Restaurants', 'Bars', 'Restaurants', 'Restaurants', 'Bars', 'Restaurants', 'Restaurants', 'Restaurants', 'Bars', 'Restaurants', 'Bars', 'Bars', 'Oil Change Stations', 'Bars', 'Bars', 'Bars', 'Pubs', 'Sports Clubs', 'Pubs', 'Restaurants', 'Bars', 'Restaurants', 'Restaurants', 'Restaurants', 'Bars']
Fashion ['Bars']
Bars ['Pubs', 'Beauty & Spas', 'Restaurants', 'Bars', 'Bars', 'Bars', 'Bars', 'Restaurants', 'Bars', 'Home & Garden', 'Bars', 'Rest

Bars []
Active Life []
Home & Garden []
Active Life []
Restaurants []
Fashion ['Restaurants']
Bars []
Pubs ['Restaurants', 'Bars']
Bars ['Restaurants', 'Restaurants', 'Restaurants', 'Restaurants', 'Bars', 'Restaurants', 'Bars']
Bars []
Restaurants []
Oil Change Stations []
Beauty & Spas []
Beauty & Spas []
Bars []
Restaurants ['Used', 'Restaurants']
Beauty & Spas []
Bars []
Home & Garden []
Restaurants ['Restaurants']
Beauty & Spas []
Bars ['Bars', 'Active Life']
Sports Clubs ['Bars']
Restaurants ['Restaurants']
Home & Garden []
Beauty & Spas []
Used []
Beauty & Spas []
Restaurants []
Home & Garden []
Beauty & Spas []
Bars []
Active Life []
Beauty & Spas []
Home Cleaning []
Bars ['Bars']
Fashion []
Beauty & Spas []
Hair Removal []
Home & Garden []
Pubs ['Bars']
Restaurants []
Restaurants []
Restaurants []
Pubs []
Beauty & Spas []
Beauty & Spas []
Bars []
Bars ['Fashion', 'Bars', 'Bars']
Home & Garden []
Restaurants ['Pubs', 'Beauty & Spas']
Bars ['Bars', 'Bars', 'Bars', 'Restaurants']


In [41]:
fromBustoBusSeries = pd.Series(index = list(G.nodes()))

In [42]:
fromBustoBusSeries = fromBustoBusSeries.reset_index()

In [43]:
def getMaxNeighbor(origin):
    maxInDegNeighbor = 0
    maxNeighbor = ""
    for neighbor in G.successors(origin):
        currentNeighborIn = G.in_degree(neighbor)
        if currentNeighborIn >=  maxInDegNeighbor:
            maxInDegNeighbor = currentNeighborIn
            maxNeighbor = neighbor
    return maxNeighbor

fromBustoBusSeries['destination'] = fromBustoBusSeries['index'].apply(getMaxNeighbor)

In [44]:
# Number of nodes in network
len(fromBustoBusSeries['index'].unique())

8553

In [45]:
# Number of nodes that have an out neighbor
len(fromBustoBusSeries[fromBustoBusSeries['destination']!=""])

2256

In [46]:
fromBustoBusSeries = fromBustoBusSeries[fromBustoBusSeries['destination']!=""].drop([0],axis = 1)

In [47]:
fromBustoBusSeries = fromBustoBusSeries.rename(columns = {'index':'current_business'})

In [48]:
fromBustoBusSeries['name_current_business'] = fromBustoBusSeries['current_business'].apply(lambda bus: Yelp_BusinessesCharlotteNAMES[bus])

In [49]:
fromBustoBusSeries['name_destination'] = fromBustoBusSeries['destination'].apply(lambda bus: Yelp_BusinessesCharlotteNAMES[bus])

In [50]:
## Save to files

In [51]:
#fromBustoBusSeries1.to_csv("APP_source_dest_Charlotte.csv")

In [52]:
fromBustoBusSeries = fromBustoBusSeries[['name_current_business','name_destination']]

In [53]:
file = "APP_source_dest_Charlotte.txt"

count1 = 0
holder1 = []

with open(file,"w") as f:
    for row in fromBustoBusSeries.iterrows(): #
        txtrow = str(row[1]['name_current_business']) +"|"+ str(row[1]['name_destination']) + "\n"
        f.writelines("%s" % txtrow)
f.close()

In [54]:
fromBustoBusSeries

,name_current_business,name_destination
0,Dillard's,LittleSpoon
4,Chick-fil-A,Cabo Fish Taco
5,The Fillmore Charlotte,VBGB Beer Hall and Garden
6,Kickstand Burgers & Bar,Lang Van Vietnamese
10,Mellow Mushroom,Soul Gastrolounge
15,Joyfully Curly,Bella Bridesmaids
20,Caliber Collision,Best Wok
24,Lorenzo's Pizzeria,Cast Iron Waffles
25,Apple Store,Amélie's French Bakery & Café
26,Coppa Coffee and Tea Cafe,Los Arcos Mexican Restaurant
